In [ ]:
from __future__ import division
from pyoperators import pcg
from pysimulators import profile

# QUBIC packages
import qubic
from qubicpack.utilities import Qubic_DataDir
from qubic.data import PATH
from qubic.io import read_map
from qubic import QubicSkySim as qss
import sys
sys.path.append('/Users/mregnier/Desktop/PhD Regnier/mypackages')


import frequency_acquisition as Acq

# Display packages
import healpy as hp
import matplotlib.pyplot as plt

# FG-Buster packages
import component_model as c
import mixing_matrix as mm

# General packages
import numpy as np
import pysm3
import warnings
from qubic import QubicSkySim as qss
import pysm3.units as u
from importlib import reload
from pysm3 import utils

from importlib import reload
import gc
from pyoperators import *
from pysimulators import *


from pysimulators.interfaces.healpy import HealpixConvolutionGaussianOperator
warnings.filterwarnings("ignore")

def integration(nu_min, nu_max, Nintegr, sky_config, d, fwhm_subbands = None):
    print(f'Integration from {nu_min:.2f} to {nu_max:.2f} GHz with {Nintegr} steps')
    obj = Acq.QubicIntegrated(d, Nsub=Nintegr, Nrec=Nintegr)
    if Nintegr == 1:
        allnus = np.array([np.mean([nu_min, nu_max])])
    else:
        allnus = np.linspace(nu_min, nu_max, Nintegr)
    m = obj.get_PySM_maps(sky_config, nus=allnus)
    
    if fwhm_subbands is not None:
        for i in range(Nintegr):
            C = HealpixConvolutionGaussianOperator(fwhm=fwhm_subbands[i])
            m[i] = C(m[i])
    
    return np.array([np.mean(m, axis=0)])


# # %config InlineBackend.figure_format='retina'
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
reload(Acq)
nside = 256
relative_bandwidth = 0.25
band = 220
band_planck = 217
noiseless = True

npointings = 1000
Nf_TOD = 4
Nf_recon = 2
fact_sub = 2

reload(Acq)


dictfilename = 'dicts/pipeline_demo.dict'
d = qubic.qubicdict.qubicDict()
d.read_from_file(dictfilename)

d['nf_recon'] = Nf_recon
d['nf_sub'] = Nf_recon*fact_sub
d['nside'] = nside
npix=12*d['nside']**2
d['RA_center'] = 0
d['DEC_center'] = -57
center = qubic.equ2gal(d['RA_center'], d['DEC_center'])
d['effective_duration'] = 3
d['npointings'] = npointings
d['tol'] = 5e-4
d['filter_nu'] = band * 1e9
d['photon_noise'] = not noiseless
d['noiseless'] = noiseless
d['config'] = 'FI'
d['filter_relative_bandwidth'] = 0.25
d['debug'] = False
d['MultiBand'] = True
d['planck'] = True
d['dtheta'] = 15
d['synthbeam_dtype'] = float

print('*************** Noise **************')
print('Noisless:      {}'.format(d['noiseless']))
print('Photon Noise:  {}'.format(d['photon_noise']))
print('************************************')

sky_config = {'dust':'d0'}

reload(Acq)

qubic_acquisition = Acq.QubicIntegrated(d, Nsub=Nf_TOD, Nrec=Nf_TOD)
qubic_acquisition_recon = Acq.QubicIntegrated(d, Nsub=fact_sub*Nf_recon, Nrec=Nf_recon)
planck_acquisition = Acq.PlanckAcquisition(band_planck, qubic_acquisition.scene)
qubicplanck_acquisition = Acq.QubicPlanckMultiBandAcquisition(qubic_acquisition, planck_acquisition)

cov = qubic_acquisition.get_coverage()
C_1degree = HealpixConvolutionGaussianOperator(fwhm = np.deg2rad(1))
#cov = C_1degree(cov)
covnorm = cov/cov.max()
seenpix = covnorm > 0.

In [ ]:
def get_pySM_maps(sky, nu):
    return np.array(sky.get_emission(nu * u.GHz, None).T * utils.bandpass_unit_conversion(nu*u.GHz, None, u.uK_CMB))

### CMB realization -> for real data, we should take Planck CMB map
cmb = qubic_acquisition.get_PySM_maps({'cmb':42}, np.array([150]))

In [ ]:
### We define foregrounds model
sky = pysm3.Sky(nside, preset_strings=['d0'])
sky_model = pysm3.Sky(nside, preset_strings=['d0'])


### We compute components, we should take Planck components for real data
nu_ref = 150
plancksky = np.zeros((Nf_TOD, 12*nside**2, 3))
skymodel = np.zeros((Nf_TOD, 12*nside**2, 3))
mysky_ref = get_pySM_maps(sky, nu_ref)
mysky_model = get_pySM_maps(sky_model, nu_ref)

In [ ]:
### We scale the components to reproduce frequency observations

comp = c.Dust(nu0=150)
beta = np.array([1.54, 20])
allnus = qubic_acquisition.allnus
sed = mm.MixingMatrix(comp).evaluator(allnus)(beta)
print(sed)

for i in range(3):
    plancksky[:, :, i] = (sed @ np.array([mysky_ref[:, i]])) + cmb[0, :, i]*0# - np.mean((sed @ np.array([mysky_ref[:, i]])), axis=0)
    skymodel[:, :, i] = sed @ np.array([mysky_model[:, i]])



k=0
for i in range(Nf_recon):
    delta = skymodel[i*fact_sub:(i+1)*fact_sub] - np.mean(skymodel[i*fact_sub:(i+1)*fact_sub], axis=0)
    for j in range(fact_sub):
        plancksky[k] -= delta[j]
        k+=1
    #plancksky[i, :, :] -= delta

### We remove the bandpass mismatch for Planck data using the same model
mean_sky = np.zeros((Nf_recon, 12*nside**2, 3))
for i in range(Nf_recon):
    mean_sky[i] = np.mean(plancksky[i*fact_sub:(i+1)*fact_sub], axis=0)

In [ ]:
plt.figure(figsize=(15, 5))

for i in range(Nf_TOD):
    hp.mollview(plancksky[i, :, 0], min=-300, max=300, sub=(1, Nf_TOD, i+1), cmap='jet', title=f'{qubic_acquisition.allnus[i]:.2f} GHz')

plt.show()

# Generate TOD

In [ ]:
### We compute QUBIC TODs with the correction of the bandpass

TOD_QUBIC = qubic_acquisition_recon.generate_tod(sky_config, noise=False, bandpass_correction=True, convolution=False)
m_sub = qubic_acquisition.get_PySM_maps(sky_config, qubic_acquisition.allnus)

In [ ]:
### We compute Planck TODs using the previous sky

TOD_PLANCK = np.zeros((Nf_recon, 12*nside**2, 3))
mrec = np.zeros((Nf_recon, 12*nside**2, 3))
for irec in range(Nf_recon):
    print(irec*fact_sub, (irec+1)*fact_sub)
    C = HealpixConvolutionGaussianOperator(fwhm = np.min(qubic_acquisition.allfwhm[irec*fact_sub:(irec+1)*fact_sub]))
    
    mrec[irec] = mean_sky[irec].copy()
    TOD_PLANCK[irec] = mean_sky[irec].copy()

R = ReshapeOperator(mrec.shape, (mrec.shape[0]*mrec.shape[1]*mrec.shape[2]))
TOD_PLANCK = R(TOD_PLANCK)

In [ ]:
### Create Planck and joint acquisition

planck_acquisition = Acq.PlanckAcquisition(band_planck, qubic_acquisition_recon.scene)
qubicplanck_acquisition = Acq.QubicPlanckMultiBandAcquisition(qubic_acquisition_recon, planck_acquisition)

In [ ]:
### Create the final TOD

n = qubicplanck_acquisition.get_noise()
TOD = np.r_[TOD_QUBIC, TOD_PLANCK] + n

In [ ]:
plt.figure(figsize=(15, 10))
plt.subplot(2, 1, 1)
plt.plot(TOD_QUBIC/TOD_QUBIC.max())
plt.ylabel('Normalized ADU', fontsize=14)

plt.subplot(2, 1, 2)
plt.plot(TOD_PLANCK/TOD_PLANCK.max())
plt.xlabel('Samples', fontsize=14)
plt.ylabel('Normalized ADU', fontsize=14)
plt.show()

In [ ]:
### We define here the final convolution and the A and b term for PCG
myfwhm = np.array([])
for i in range(Nf_recon):
    myfwhm = np.append(myfwhm, np.sqrt(qubic_acquisition.allfwhm[i*fact_sub:(i+1)*fact_sub]**2 - np.min(qubic_acquisition.allfwhm[i*fact_sub:(i+1)*fact_sub]**2)))

### Reconstruction operator

index = np.where(seenpix == True)[0]
mask = np.ones(12*nside**2)
mask[index] /= 1e6

H = qubicplanck_acquisition.get_operator(convolution=False, myfwhm=None)
invN = qubicplanck_acquisition.get_invntt_operator()

### Putting the Planck weigth inside QUBIC patch to 0
#for i in range(Nf_recon):
#    invN.operands[i+1].operands[1] *= 1e6

R = ReshapeOperator((1, 12*nside**2, 3), (12*nside**2, 3))

### Solve PCG
U = (
    ReshapeOperator((Nf_recon * sum(seenpix) * 3), (Nf_recon, sum(seenpix), 3)) *
    PackOperator(np.broadcast_to(seenpix[None, :, None], (Nf_recon, seenpix.size, 3)).copy())
).T

### Compute A and b
with rule_manager(none=True):
    if Nf_recon == 1:
        A = U.T * R.T * H.T * invN * H * R * U
        x_planck = mean_sky * (1 - seenpix[None, :, None])
        b = U.T ( R.T * H.T * invN * (TOD - H(R(x_planck))))
    else:
        A = U.T * H.T * invN * H * U
        x_planck = mean_sky * (1 - seenpix[None, :, None])
        b = U.T (  H.T * invN * (TOD - H(x_planck)))
        
M = Acq.get_preconditioner(np.ones(12*nside**2))

In [ ]:
### PCG
solution_qubic_planck = pcg(A, b, x0=None, M=M, tol=1e-25, disp=True, maxiter=100)

In [ ]:
#k=0
#for irec in range(Nf_recon):
#    delta = m_sub[fact_sub*irec:(irec+1)*fact_sub] - np.mean(m_sub[fact_sub*irec:(irec+1)*fact_sub], axis=0)
#    for jfact in range(fact_sub):
#        m_sub[k] -= delta[jfact]
#        k+=1

In [ ]:
istk = 1

v = 8
r = 8
res = 25

plt.figure(figsize=(10, 8))

k=0
for i in range(Nf_recon):
    print(i, i*fact_sub, (i+1)*fact_sub)
    
    ### Inputs
    C = HealpixConvolutionGaussianOperator(fwhm=0)#np.min(qubic_acquisition.allfwhm[i*fact_sub:(i+1)*fact_sub]))#allfwhm_ref[i])
    myinputs = np.mean(m_sub, axis=0).copy() * 0
    seenpix = covnorm > 0
    myinputs[seenpix] = np.mean(m_sub[i*fact_sub:(i+1)*fact_sub], axis=0)[seenpix]
    myinputs = C(myinputs)
    
    ### Outputs
    mysolution = myinputs.copy()
    mysolution[seenpix] = solution_qubic_planck['x'][i].copy()
    
    seenpix = covnorm > 0
    myinputs[~seenpix] = hp.UNSEEN
    mysolution[~seenpix] = hp.UNSEEN
    \
    hp.gnomview(myinputs[:, istk], cmap='jet', rot=center, reso=res, min=-v, max=v, sub=(Nf_recon, 3, k+1), title='Input')
    hp.gnomview(mysolution[:, istk], cmap='jet', rot=center, reso=res, min=-v, max=v, sub=(Nf_recon, 3, k+2), title='Output')
    resi = (mysolution[:, istk]-myinputs[:, istk])
    resi[~seenpix] = hp.UNSEEN
    hp.gnomview(resi, cmap='jet', rot=center, reso=res, title=f'Residual - RMS : {np.std(resi[seenpix]):.3e}', min=-r, max=r, sub=(Nf_recon, 3, k+3))

    k+=3
    
plt.show()